In [2]:
from ScrapeFB import *
import pandas as pd
import selenium
import bs4, time, re, json, datetime

In [2]:
#URL = 'https://www.facebook.com/pg/set.or.th/posts/'
URL = 'https://www.facebook.com/SCGofficialpage/'


In [4]:
for i in range(2):
    window.scrape(filename='scgtest.json', scroll=2)

100%|██████████| 2/2 [00:20<00:00, 10.30s/it]


# mobile FB test

In [6]:
URL = 'https://m.facebook.com/AIS/posts'
window = WindowNoLogin(url=URL, headless=False, browser='firefox')

In [8]:
elem = window.driver.find_elements_by_css_selector('article')[5]
soup = bs4.BeautifulSoup(elem.get_attribute('innerHTML'))
for p in soup.find_all('p'):
    for elem in p.children:
        print(elem)

เอไอเอส พอยท์ แลกล้านความสุข ฉลองต้อนรับปีใหม่!!
<br/>
 ลูกค้าเอไอเอส ใช้พอยท์กับทุกเรื่องที่ชอบ หลากหลายความพิเศษ เช็กพอยท์ แล้วใช้เลย! ที่แอปmyAIS คลิกเลย! 
<a data-sigil="MLynx_asynclazy" href="https://lm.facebook.com/l.php?u=https%3A%2F%2Fm.ais.co.th%2FQYH0nPQTx&amp;h=AT1lDvfMtNrExheOA4ep6r-2CV9REa9OTpDkoA4DRpEnUjkOcQcfSYdemN_bYfUL8BOcOAD9P3WP7IpJ-j6xqrqHVr0x2bnomcNg7k9Q0Lu7YOhJonHVj34M7tYkIegIAx76AE4ba7rxM1sQQLFok6b31rFWFlmszGzv37P_zwtQ" rel="noopener" target="_blank">https://m.ais.co.th/QYH0nPQTx</a>
 
<br/>
 ·
<span class="text_exposed_hide">… <span data-sigil="more"><a href="/story.php?story_fbid=10159692559016554&amp;id=127434041553">เพิ่มเติม</a></span></span>
<span class="text_exposed_show"> แลกสิทธิ์ลุ้นสร้อยคอทองคำหนัก 2 บาท 100 รางวัล!! <br/> · แลกสติกเกอร์ไลน์ 5 ลายใหม่ สุดฮิต!!<br/> · แลกความหวาน รับปีใหม่ที่ After you ทุกสาขาที่ร่วมรายการ<br/> · แลกความอร่อย ที่ฟู้ดคอร์ท Central และ The mall ที่ร่วมรายการทั่วประเทศ<br/> =============<br/> • สมัครสะสมหรือเช็กคะแนน กด*55

In [9]:
def get_article(soup):
    article = ''
    hashtags = []
    for p in soup.find_all('p'):
        for elem in p.children:
            if type(elem) == bs4.element.NavigableString:
                article += elem
            elif elem.name == 'br':
                article += '\n'
            elif elem.name == 'a':
                if ['_5ayv','_qdx'] == elem.attrs.get('class',[]):
                    hashtags.append(elem.text)
                else:
                    article += elem.text
            elif elem.name == 'span':
                if 'text_exposed_hide' in elem.attrs.get('class', []):
                    continue
                elif 'text_exposed_show' in elem.attrs.get('class', []):
                    for e in elem:
                        if type(e) == bs4.element.NavigableString:
                            article += e.lstrip()
                        elif e.name == 'br':
                            article += '\n'
                        elif e.name == 'a':
                            if ['_5ayv','_qdx'] == e.attrs.get('class',[]):
                                hashtags.append(e.text)
                            else:
                                article += e.text
                else: # e.g. emoji
                    try:
                        article += elem.text
                    except:
                        pass
        article += '\n\n'
        article = re.sub('\n +', '\n', article).strip()
    return article, hashtags

In [10]:
def get_images(soup):
    images = []
    try: 
        img_div = soup.select_one('._5rgu._7dc9._27x0')
        for img in img_div.select('img'):
            images.append(img.get('src'))
    except:
        pass
    return images

In [27]:
def get_video(soup):
    try: 
        img_div = soup.select_one('._5rgu._7dc9._27x0')
        video_div = img_div.select_one('div[data-sigil="inlineVideo"]')
        return re.search(r'src":"(.+?)"', video_div.get('data-store')).group(1)
    except:
        return None

In [39]:
def get_reaction(soup):
    footer = str(soup.find('footer'))
    try:
        like = re.search(r'(?:ถูกใจ )?(\d+) (?:คน|Like)', footer).group(1)
    except:
        like = 0
    try:
        comment = re.search(r'(?:ความคิดเห็น )?(\d+) (?:รายการ|comment)', footer).group(1)
    except:
        comment = 0
    try:
        share = re.search(r'(?:แชร์ )?(\d+) (?:ครั้ง|share)', footer).group(1)
    except:
        share = 0
    return like, comment, share


In [40]:
elem = window.driver.find_elements_by_css_selector('article')[12]
soup = bs4.BeautifulSoup(elem.get_attribute('innerHTML'))
print(get_article(soup))

('💼  ชาวออฟฟิศอย่างเรา ๆ หลังก็ปวด งานก็ต้องทำ จะมีอะไรช่วยได้บ้างมั้ยนะ?\n.\nจุ๊ ๆ อย่าเสียงดังไป 🤫  วันนี้แอดมีไอเทมเด็ดช่วยเสริมดวงให้ชาวออฟฟิศมาฝากนั่นก็คือ “เบอร์เสริมชีวิต สุขภาพดี งานเด่น”ช่วยอัปดวงแบบก้าวกระโดดให้งานรุ่ง ชีวิตราบรื่น เจ้านายรัก เจ้านายหลง แถมได้สุขภาพดี ไร้โรคภัยไข้เจ็บ \n.\nดีแบบนี้ อย่าลืมมาเปลี่ยนเบอร์กันนะคร้าบ \nคลิก https://m.ais.co.th/DeHrGoAU8\n.', ['#AIS5G'])


In [41]:
footer = str(soup.find('footer'))
get_reaction(soup)

('56', '5', '2')

In [35]:
footer

'<footer class="_22rc" data-ft=\'{"tn":"*W"}\'><div class="_2ip_ _4b44" data-sigil="mufi-inline" id="feedback_inline_10159682880921554"><div class="_2og4 slimLike _15kj _34qc _4b45" id="counts_feedback_inline_10159682880921554"><a data-sigil="feed-ufi-trigger" href="/story.php?story_fbid=10159682880921554&amp;id=127434041553&amp;__tn__=%2AW-R"><span class="_28-d" id="u_6_d"><span class="like_def _28wy">ถูกใจ 56 คน</span><span class="cmt_def _28wy">ความคิดเห็น 5 รายการ</span></span><span class="_28wy">แชร์ 2 ครั้ง</span></a></div><div class="_52jh _5909 _15kk _15ks _4b47" data-sigil="ufi-inline-actions"><div class="_7om2"><div class="_52jj _15kl _4g34"><a class="_15kr _77li" data-click=\'{"event":"click_share_ufi","target_id":"226547524203272:10159682880926554"}\' data-ft=\'{"tn":"J"}\' data-sigil="share-popup" data-store=\'{"is_acting_as_page":false,"reshare_post":true,"share_id":"10159682880926554","feedback_source":0,"feedback_referrer":null,"internal_preview_image_id":null,"shareabl

# mobile FB

In [1]:
from ScrapeFBMobile import *
import pandas as pd
import selenium
import bs4, time, re, json, datetime

In [2]:
URL = 'https://m.facebook.com/AIS/posts'
window = Window(url=URL, headless=False, browser='firefox')

In [6]:
for i in range(2):
    try:
        window.scrape(filename='aisFB.json', scroll=5)
    except Exception as e:
        print(e)
    time.sleep(5)

100%|██████████| 5/5 [00:05<00:00,  1.13s/it]


In [5]:
window.write('aisFB.json')

# scroll manually

In [22]:
window.append_data()

In [23]:
window.delete_elements()

In [24]:
window.write(savefile)

# test

In [83]:
elem = window.driver.find_elements_by_css_selector('.du4w35lb.k4urcfbm.l9j0dhe7.sjgh65i0')[1]
raw_html = elem.get_attribute('innerHTML') # get html content of element
soup = BeautifulSoup(raw_html, features='html.parser') 

In [93]:
for paragraph in soup.select('.cxmmr5t8.oygrvhab.hcukyx3x.c1et5uql.ii04i59q'):
    print('-'*50)
    for div in paragraph.select('div'):
        for child in div.children:
            print(child, type(child),'\n')

--------------------------------------------------
ไม่ว่าใคร ก็ช่วยลดโลกร้อนได้ ! <class 'bs4.element.NavigableString'> 

. <class 'bs4.element.NavigableString'> 

วันนี้จะพาเพื่อน ๆ ไปดู 8 วิธีลดโลกร้อน ของทัพนักกีฬาระดับโลก ที่รับรองว่าเพื่อน ๆ สามารถนำไปปรับใช้ทำตามได้แน่นอนครับ ! <class 'bs4.element.NavigableString'> 

. <class 'bs4.element.NavigableString'> 

ชอบไอเดียลดโลกร้อนแบบไหน อย่าลืม!! มาชาเลนจ์กันได้ที่  <class 'bs4.element.NavigableString'> 

TikTok Challenge #ลดโลกร้อนเริ่มที่เรา  <class 'bs4.element.NavigableString'> 

ลุ้นรับ iPad Air และกระเป๋าถุงปูน “คิด-จาก-ถุง”  <class 'bs4.element.NavigableString'> 

คลิก ! :  <class 'bs4.element.NavigableString'> 

<span><a class="oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl py34i1dx gpro0wi8" href="https://l.facebook.com/l.php?u=https%3A%2F%2Fvt.tiktok.com%2FZSqaYd3k%2F%3Ffbcli

In [95]:
article = []
hashtags = []
for paragraph in soup.select('.cxmmr5t8.oygrvhab.hcukyx3x.c1et5uql.ii04i59q'):
    text_in_paragraph = ''
    for div in paragraph.select('div'):
        for child in div.children:
            if type(child) == bs4.element.NavigableString:
                text_in_paragraph += '\n' + child
            elif type(child) == bs4.element.Tag:
                inner_text = child.text.strip()
                if inner_text != '': # hashtag or link
                    if inner_text.startswith('#'): # hashtag
                        hashtags.append(inner_text)
                    else: # link
                        text_in_paragraph += inner_text
                else:
                    emoji = re.search(r'<img alt="(.+?)".+/?>', str(child))
                    if emoji:
                        text_in_paragraph += emoji.group(1)
    article.append(text_in_paragraph)
print('\n\n'.join(article))
print(hashtags)


ไม่ว่าใคร ก็ช่วยลดโลกร้อนได้ !
.
วันนี้จะพาเพื่อน ๆ ไปดู 8 วิธีลดโลกร้อน ของทัพนักกีฬาระดับโลก ที่รับรองว่าเพื่อน ๆ สามารถนำไปปรับใช้ทำตามได้แน่นอนครับ !
.
ชอบไอเดียลดโลกร้อนแบบไหน อย่าลืม!! มาชาเลนจ์กันได้ที่ 
TikTok Challenge #ลดโลกร้อนเริ่มที่เรา 
ลุ้นรับ iPad Air และกระเป๋าถุงปูน “คิด-จาก-ถุง” 
คลิก ! : https://vt.tiktok.com/ZSqaYd3k/
.
มาร่วมหาวิธีลดโลกร้อนเพิ่มเติม 
คลิก ! : http://bit.ly/FBSCGCES441
.
 
 
['#ลดโลกร้อนเริ่มที่เรา', '#SCG', '#ClimateEmergency']


# check file

In [13]:
pd.read_json(savefile).drop_duplicates('url')
#pd.read_json(savefile)

,date,article,url,emotion,comment,share,hashtags
0,2020-09-14 03:00:00,"หากยังรู้สึกว่า ""อากาศร้อนขึ้นทุกปี"" โลกร้อน ก...",https://www.facebook.com/3978014498881647,"{'like': 139, 'love': 5, 'care': 2, 'haha': 0,...",11,50,"[ลดโลกร้อนเริ่มที่เรา, SCG]"
1,2020-12-02 02:07:00,อดทน รับผิดชอบ มีวินัย 3 แนวทางที่เราช่วยกันได...,https://www.facebook.com/4224292444253850,"{'like': 14, 'love': 0, 'care': 0, 'haha': 0, ...",0,1,[]
2,2020-12-02 00:52:00,เวลาพูดถึงการใช้ชีวิตที่ 'เป็นมิตรกับสิ่งแวดล้...,https://www.facebook.com/4224165540933207,"{'like': 27, 'love': 0, 'care': 0, 'haha': 0, ...",0,2,"[SCG, SDSymposium2020, ร่วมมือเพื่อโลก, Tansir..."
3,2020-12-01 19:00:00,ลดโลกร้อนไม่ใช่เรื่องไกลตัว สามารถทำได้หลายวิธ...,https://www.facebook.com/4163090247040737,"{'like': 54, 'love': 0, 'care': 0, 'haha': 0, ...",0,3,"[ลดโลกร้อนเริ่มที่เรา, SCG, ClimateEmergency, ..."
4,2020-12-01 04:30:00,อัปเดต Highlight เรื่องราวสำคัญเดือนพฤศจิกายน ...,https://www.facebook.com/4221510804532014,"{'like': 114, 'love': 4, 'care': 0, 'haha': 0,...",2,6,[]
...,...,...,...,...,...,...,...
2881,2016-02-09 03:55:00,ยังมีเวลาสำหรับเพื่อนๆน้องๆ ที่อยากร่วมเป็นส่ว...,https://www.facebook.com/1268282656521525,"{'like': 83, 'love': 0, 'care': 0, 'haha': 0, ...",0,5,[]
2882,2016-02-04 01:56:00,เพื่อนๆหลายคนน่าจะชอบปั่นจักรยานกันไม่ว่าจะไปท...,https://www.facebook.com/1265195363496921,"{'like': 47, 'love': 0, 'care': 0, 'haha': 0, ...",0,3,[]
2883,2016-02-01 03:18:00,สำหรับพ่อแม่ที่ต้องการทำกิจกรรมแต่ต้องดูแลลูกน...,https://www.facebook.com/1263363030346821,"{'like': 53, 'love': 0, 'care': 0, 'haha': 0, ...",0,2,[]
2884,2016-01-29 03:33:00,เอสซีจีแถลงผลประกอบการปี 2558 เดินหน้าลงทุนต่อ...,https://www.facebook.com/1261482087201582,"{'like': 294, 'love': 0, 'care': 0, 'haha': 0,...",0,12,[]
